In [10]:
import glob
from pathlib import Path
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import dateparser

path = "../Tripadvisor_web_scraper/data/*.csv"
dataframe = pd.DataFrame()
for fname in glob.glob(path):
    x = pd.read_csv(fname, low_memory=False)
    x = x.dropna(subset=['date'])
    x['date'] = [date.replace('Erlebnisdatum: ', '') for date in x['date']]
    x['date'] = [dateparser.parse(date).strftime('%Y-%m')  for date in x['date']]
    x['place'] = Path(fname).stem
    x['visit'].fillna('', inplace=True)
    x['visit'] = [visit_type.replace('Reiseart: ', '') for visit_type in x['visit']]
    x = x[['date', 'place', 'rating', 'visit']]
    dataframe = pd.concat([dataframe, x], axis=0)

df = dataframe.groupby(['date', 'place'], as_index=False)[['rating']].mean()
df2 = dataframe.groupby(['date', 'place'])[['date']].count()
df['#_of_visits'] = df2['date'].values
df['city_district'] =df['place']

city_district = ['Altstadt-Lehel', 'Ludwigsvorstadt-Isarvorstadt', 'Maxvorstadt', 'Schwabing-West'
    , 'Au-Haidhausen'
    , 'Sendling'
    , 'Sendling-Westpark'
    , 'Schwanthalerhöhe'
    , 'Neuhausen-Nymphenburg'
    , 'München-Moosach'
    , 'Milbertshofen-Am Hart'
    , 'Schwabing-Freimann'
    , 'Bogenhausen'
    , 'Berg am Laim'
    , 'Trudering-Riem'
    , 'Ramersdorf-Perlach'
    , 'Obergiesing'
    , 'Untergiesing-Harlaching'
    , 'Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln'
    , 'Hadern'
    , 'Pasing-Obermenzing'
    , 'Aubing-Lochhausen-Langwied'
    , 'Allach-Untermenzing'
    , 'Feldmoching-Hasenbergl'
    , 'Laim']
geolocator = Nominatim(user_agent='salut', timeout=3)

places = df['place'].unique()
y = {}
for place in places:
    try:
        print(place)
        location = geolocator.geocode(place, addressdetails=True, country_codes='de')
        location2 = location.address
        for district in city_district:
            if district in location2:
                y[place] = district
    except:
        y[place] = ''

for index, row in df.iterrows():
        df['city_district'][index] = y[row['place']]

nparray = np.array(df)
ret = []

for place in places:
    name = place
    tmp = pd.DataFrame(df[df.place == name])
    # Change format of date column
    #tmp['date'] = pd.to_datetime(tmp['date'])
    tmp = tmp.set_index('date')
    mat = pd.DataFrame(index= tmp.index)
    del tmp['place']
    mat[place] = tmp.apply(lambda x: [np.array(x)], axis=1).apply(lambda x: x[0])
    ret.append(mat)


# Concat the feature table

df_clean = pd.concat(ret, axis=1, sort=True)


Bayerisches Nationalmuseum
Neue Pinakothek
Bayerisches Staatsorchester
Museum Brandhorst
MÃ¼nchner Stadtmuseum
Nationaltheater
Residenztheater
Olympiastadion
Alte Pinakothek
Museum Mensch und Natur
MÃ¼nchner Philharmoniker
Prinzregententheater
Pinakothek der Moderne
StÃ¤dtische Galerie im Lenbachhaus
Deutsches Museum
Tierpark Hellabrunn
Olympiapark
Theater am Gärtnerplatz


C:\Users\steph\anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
df_clean

,Bayerisches Nationalmuseum,Neue Pinakothek,Bayerisches Staatsorchester,Museum Brandhorst,MÃ¼nchner Stadtmuseum,Nationaltheater,Residenztheater,Olympiastadion,Alte Pinakothek,Museum Mensch und Natur,MÃ¼nchner Philharmoniker,Prinzregententheater,Pinakothek der Moderne,StÃ¤dtische Galerie im Lenbachhaus,Deutsches Museum,Tierpark Hellabrunn,Olympiapark,Theater am Gärtnerplatz
2010-03,"[40.0, 1, Altstadt-Lehel]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07,NaN,"[50.0, 1, Maxvorstadt]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09,NaN,NaN,"[50.0, 1, ]","[40.0, 1, Maxvorstadt]","[50.0, 1, ]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12,NaN,"[50.0, 1, Maxvorstadt]",NaN,"[50.0, 1, Maxvorstadt]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01,"[50.0, 1, Altstadt-Lehel]",NaN,"[50.0, 2, ]",NaN,"[40.0, 1, ]","[50.0, 1, Altstadt-Lehel]","[50.0, 1, Altstadt-Lehel]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03,"[40.0, 2, Altstadt-Lehel]",NaN,"[50.0, 1, ]",NaN,"[40.0, 1, ]",NaN,NaN,"[50.0, 1, Milbertshofen-Am Hart]",NaN,NaN,NaN,NaN,"[40.0, 1, Maxvorstadt]",NaN,"[50.0, 2, Ludwigsvorstadt-Isarvorstadt]",NaN,"[50.0, 1, Milbertshofen-Am Hart]",NaN
2020-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[50.0, 1, Milbertshofen-Am Hart]",NaN
2020-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[25.0, 2, Untergiesing-Harlaching]",NaN,NaN
2020-06,NaN,NaN,NaN,NaN,"[10.0, 1, ]",NaN,NaN,"[50.0, 2, Milbertshofen-Am Hart]","[45.0, 4, Maxvorstadt]",NaN,NaN,NaN,"[40.0, 1, Maxvorstadt]","[50.0, 2, ]",NaN,"[30.0, 8, Untergiesing-Harlaching]","[50.0, 1, Milbertshofen-Am Hart]",NaN


In [16]:
# convert the time-series to an array of sequences
def create_sequences(data, timesteps=1, dropNa=True):
    # drop row's which include Nan elements (data preprocessing)
    df = pd.DataFrame(data)
    if dropNa == True:
        df.dropna(inplace=True)
    data = df.values
    # create x and y out of data
    dataX, dataY = [], []
    for i in range(len(data)):
        endIdx = i + timesteps
        # stop if reached the end of data
        if endIdx + 1 > len(data):
            break
        dataX.append(data[i:endIdx, :])
        dataY.append(data[endIdx, :])
    return np.array(dataX), np.array(dataY)

In [30]:
pd.DataFrame(df_clean['Bayerisches Nationalmuseum'])

,Bayerisches Nationalmuseum
2010-03,"[40.0, 1, Altstadt-Lehel]"
2010-07,NaN
2010-09,NaN
2010-12,NaN
2011-01,"[50.0, 1, Altstadt-Lehel]"
...,...
2020-03,"[40.0, 2, Altstadt-Lehel]"
2020-04,NaN
2020-05,NaN
2020-06,NaN


In [35]:
a = [[1,2],[3,4],[5,6],[7,8],[9,10]]

In [49]:
# create x and y array
timesteps = 2
#trainX, trainY = create_sequences(df_clean['Bayerisches Nationalmuseum'], timesteps)
trainX, trainY = create_sequences(a, timesteps)
# the input should now have the shape: [samples, timesteps, features]
print('Input shape:', trainX.shape)
print('Output shape:', trainY.shape)

Input shape: (3, 2, 2)
Output shape: (3, 2)


In [50]:
for i in range(3):
    print('Input: ','\n',trainX[i],'\n','Output: ','\n',trainY[i],'\n')

Input:  
 [[1 2]
 [3 4]] 
 Output:  
 [5 6] 

Input:  
 [[3 4]
 [5 6]] 
 Output:  
 [7 8] 

Input:  
 [[5 6]
 [7 8]] 
 Output:  
 [ 9 10] 

